# Neptune + XGBoost

## Install dependencies

In [ ]:
! pip install neptune-client==0.9.16 xgboost==1.4.0 neptune-xgboost==0.9.8 scikit-learn==0.24.2 psutil==5.8.0 graphviz==0.16

## Quickstart

### Import libraries

In [ ]:
import xgboost as xgb
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

### Create run

In [ ]:
import neptune.new as neptune

run = neptune.init(
    project="common/xgboost-integration",
    api_token="ANONYMOUS",
    name="xgb-train",
    tags=["xgb-integration", "train"],
)

Link above is a link to the run. Click on it, and leave the run tab open - you will get back to it when you start model training.

Few explanations:
1. You need to pass project to the `project`, parameter to inform Neptune where to log metadata. Project is a string of this form: `my_workspace/my_project`.
1. There are more parameters to customize Neptune behavior, check [neptune.init() docs](https://docs.neptune.ai/api-reference/neptune#init) for more details.

----

**Note**

Instead of logging data to the public project `"common/xgboost-integration"` as an anonymous user `"neptuner"` you can log it to your own project.

To do that:
1. Follow the [installation and setup](https://docs.neptune.ai/getting-started/installation) that will show you how to use individual, private api_token.
1. Create new [private project](https://docs.neptune.ai/administration/workspace-project-and-user-management/projects).
1. Pass this project name here, instead of `"common/xgboost-integration"`.

At this point you will be ready to log XGBoost runs to your own project :)

### Create neptune callback

In [ ]:
from neptune.new.integrations.xgboost import NeptuneCallback

neptune_callback = NeptuneCallback(run=run, log_tree=[0, 1, 2, 3])

This callback will do metadata logging during training. You will pass it to the XGBoost `train()` function.

It also works with `cv()` function and sklearn-like API of the XGBoost (in this case you pass it to the `model.fit()` function).

----

Notice that you will log trees with indices `[0, 1, 2, 3]`.

### Prepare data and define parameters

In [ ]:
# Prepare data
X, y = load_boston(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_test, label=y_test)

# Define parameters
model_params = {
    "eta": 0.7,
    "gamma": 0.001,
    "max_depth": 9,
    "objective": "reg:squarederror",
    "eval_metric": ["mae", "rmse"]
}
evals = [(dtrain, "train"), (dval, "valid")]
num_round = 57

### Train the model and log metadata to the run in Neptune

In [ ]:
xgb.train(
    params=model_params,
    dtrain=dtrain,
    num_boost_round=num_round,
    evals=evals,
    callbacks=[
        neptune_callback,
        xgb.callback.LearningRateScheduler(lambda epoch: 0.99**epoch),
        xgb.callback.EarlyStopping(rounds=30)
    ],
)

This cell do the model trainig and metadata logging to Neptune.

### Analyze logged metadata in the Neptune App

Go to the run link and explore metadata (metrics, all parameters, learning rate, pickled model, visualizations) that were logged to the run in Neptune.

Link should look like this:

https://app.neptune.ai/o/common/org/xgboost-integration/e/XGBOOST-84/all?path=training